In [2]:
import pandas as pd
import json
import requests
import pickle

In [ ]:
from bs4 import BeautifulSoup
import requests

## Acquire 8 exotic cocktail recipes via scraping

In [363]:


exotic_cocktails_url = 'https://www.pubclub.com/lifestyle/8-exotic-cocktails-from-around-the-world-that-you-must-try-at-your-next-party/'
cocktails_filepath = 'cocktail_recipe_sources/'
exotic_cocktails_filename = '8%20Exotic%20Cocktails%20From%20Around%20The%20World%20That%20You%20Must%20Try%20At%20Your%20Next%20Party.html'
#exotic_cocktails_response = requests.get(exotic_cocktails_url)
#print( exotic_cocktails_response.status_code )
#exotic_cocktails_page = exotic_cocktails_response.text
#soup = BeautifulSoup(exotic_cocktails_page, "lxml")


with open(cocktails_filepath + '8_exotic_cocktail_recipies.html') as fp:
    soup = BeautifulSoup(fp, "lxml")

In [364]:
cocktail_names = soup.find_all('h3', class_='western')
#cocktails = soup.find_all('h4', class_='western')
#for c in cocktails:
cocktail_dict = {}
for c_name in cocktail_names:
    cocktail_name = c_name.text
    #print(cocktail_name)
    c = c_name.find_next_sibling('p').find_next_sibling('h4', class_='western')
    
    if c.text =='Ingredients':
        #print( c )
        ul = c.find_next_sibling('p').find_next_sibling('ul')
        ing_list = ul.find_all('li')
        list_of_ingredients = []
        for ing in ing_list:
            #print( ing )
            list_of_ingredients.append(ing.text.strip())
    cocktail_dict[cocktail_name] = list_of_ingredients    
            
print( cocktail_dict )   

{'1.Rosé All Day Cocktail': ['Bottle Rosé', 'Vodka', 'Blackberries', 'Lime Juice', 'Honey', 'Lime', 'Raspberries', 'Strawberries', 'Ice'], '2.Lychee Martini': ['Ice cubes', 'vodka', 'lychee juice', 'orange bitters', 'lychees', 'lemon juice'], 'Preparation': ['rhubarb stalks', 'frozen raspberries', 'raw sugar cane', 'lemon juice'], '3.Raspberry-Rhubarb Collins Punch': ['rhubarb stalks', 'frozen raspberries', 'raw sugar cane', 'lemon juice'], '4.Turmeric Colada': ['Ice', 'white rum', 'turmeric cordial', 'coconut milk', 'pineapple juice', 'lime juice', 'simple syrup', 'pineapple.'], '5.PINA COLADA': ['Ice', 'white rum', 'dark rum', 'coconut cream', 'double cream or whipping cream', 'pineapple juice', 'Dash of angostura bitters', 'Maraschino cherries', 'Pineapple slices'], '6.Fruity Margarita Punch': ['fruit punch', 'tequila', 'triple sec (Cointreau)', 'lime juice', 'Oranges', 'limes'], '7.Watermelon Lemonade Margarita': ['tequila', 'triple sec', 'Midori', 'watermelon juice', 'lemonade']}


In [365]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

In [366]:
exotic_c_word_list = []

for ct in cocktail_dict:
     exotic_c_word_list.extend( cocktail_dict[ct] )

In [367]:
exotic_c_word_list = [lemmatizer.lemmatize(s.lower()) for s in exotic_c_word_list]
exotic_c_word_list

['bottle rosé',
 'vodka',
 'blackberry',
 'lime juice',
 'honey',
 'lime',
 'raspberry',
 'strawberry',
 'ice',
 'ice cubes',
 'vodka',
 'lychee juice',
 'orange bitters',
 'lychee',
 'lemon juice',
 'rhubarb stalks',
 'frozen raspberries',
 'raw sugar cane',
 'lemon juice',
 'rhubarb stalks',
 'frozen raspberries',
 'raw sugar cane',
 'lemon juice',
 'ice',
 'white rum',
 'turmeric cordial',
 'coconut milk',
 'pineapple juice',
 'lime juice',
 'simple syrup',
 'pineapple.',
 'ice',
 'white rum',
 'dark rum',
 'coconut cream',
 'double cream or whipping cream',
 'pineapple juice',
 'dash of angostura bitters',
 'maraschino cherries',
 'pineapple slices',
 'fruit punch',
 'tequila',
 'triple sec (cointreau)',
 'lime juice',
 'orange',
 'lime',
 'tequila',
 'triple sec',
 'midori',
 'watermelon juice',
 'lemonade']

In [368]:
with open("data/" + "exotic_c_word_list.pickle", 'wb') as to_write:
    pickle.dump(exotic_c_word_list, to_write)

## Acquire 500+ cocktail recipes from the CocktailDB JSON source

In [369]:
drink_url_stem ='https://www.thecocktaildb.com/api/json/v2/9973533/lookup.php?i='
alcohol_api_url = 'https://www.thecocktaildb.com/api/json/v2/9973533/filter.php?a=Alcoholic'

In [370]:
drink_url_stem ='https://www.thecocktaildb.com/api/json/v2/9973533/lookup.php?i='

def cocktail_list_from_cocktail_db():
    f=alcohol_api_url
    data = requests.get(f)    
    parsed_json = json.loads(data.text)   
    drink_dict_list = []
    
    for i in (parsed_json["drinks"]):
        drink_id = i["idDrink"]
        drink_url = drink_url_stem + drink_id
        r_data = requests.get(drink_url)
        drink_dict = r_data.json()['drinks'][0]
        drink_dict_list.append(drink_dict)
    return( drink_dict_list )

cocktail_db_list = cocktail_list_from_cocktail_db() 
#print( cocktail_db_list )  

with open("data/" + "cocktail_db_list.pickle", 'wb') as to_write:
    pickle.dump(cocktail_db_list, to_write)

In [371]:
with open("data/" + "cocktail_dict.pickle", 'wb') as to_write:
    pickle.dump(cocktail_dict, to_write)